<div id="container" style="position:relative;">
<div style="float:left"><h1> Market Basket Analysis </h1></div>
<div style="position:relative; float:right"><img style="height:65px" src ="https://twomagpiesbakery.co.uk/wp-content/uploads/2020/11/logo-no-site.jpg" />
</div>
</div>

**Steps to determine market analysis update.**

**UPDATE**

Mining retail datasets like this is done to find a number of relations:

Complementary products: products which are often bought together, like chips and salsa
Substitute products: products which replace each other, like Coke and Pepsi
Trigger products: products which when bought, trigger other purchases
Common Baskets: combinations of products that are often bought together 


Hypothesis : business suspect X shop sells X more / less, hard to predict sales for this products etc and from EDA tea and coffee with X,y,z

---

## Importing Libraries

In [1]:
# Main libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
import os
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


from plotly.subplots import make_subplots
from pandas.tseries.offsets import DateOffset

## Importing Custom Functions

In [2]:
import BakeryFunctions as bakery

In [3]:
def transaction_encoder(df):
    df = df.groupby('TransactionId')['ProductName'].agg(['unique'])
    df = df['unique'].values.tolist()
    te = TransactionEncoder()
    df = te.fit_transform(df)
    df = pd.DataFrame(ald, columns=te.columns_)
    return df

## To Get Current Directory

In [4]:
working_directory = os.getcwd()
working_directory

'/Users/abimagnall/Documents/BrainStation/Capstone/Data'

## Importing Preprocessed Dataset

In [5]:
aldeburgh = pd.read_csv(working_directory+'/2_clean_data/aldeburgh_clean.csv')
southwold = pd.read_csv(working_directory+'/2_clean_data/southwold_clean.csv')
darsham = pd.read_csv(working_directory+'/2_clean_data/darsham_clean.csv')
norwich = pd.read_csv(working_directory+'/2_clean_data/norwich_clean.csv')

products = pd.read_csv(working_directory+'/3_preprocessed_data/PreprocessedProducts.csv')

# Data Processing
The datasets need to be in the correct format, with prodcut names for the 20 hero products that were selected by the business that require further analysis into, along with the `TransactionId` and the `Quantity`.

To do this, first the tables are merged using the `merge_dataset` function. 

In [6]:
aldeburgh = bakery.merge_dataset(aldeburgh, products, 'ProductId')
southwold = bakery.merge_dataset(southwold, products, 'ProductId')
darsham = bakery.merge_dataset(darsham, products, 'ProductId')
norwich = bakery.merge_dataset(norwich, products, 'ProductId')

In [7]:
# To validate it worked 
aldeburgh.head()

,TransactionId,ProductId,UnitPrice,Quantity,DiscountAmount,DiscountReasonId,RefundReasonId,Date,Name,ProductName,Eat In / Take Out,Category
0,325356083,21567882,3.2,1,0.00,0.0,0.0,2020-09-01,BLUEBERRY & LEMON CAKE T/O,Remove,Remove,Remove
1,325399103,1677432,2.6,1,0.00,0.0,0.0,2020-09-01,FLAT WHITE REG E/I,Coffee,E/I,Hot Drink
2,325400582,20505962,4.5,1,2.25,13524.0,0.0,2020-09-01,SPELT & CHIA,Remove,Remove,Remove
3,325400582,15137944,1.8,1,0.00,0.0,0.0,2020-09-01,AMARETTI T/O,Remove,Remove,Remove
4,325403315,20405549,3.4,1,0.00,0.0,0.0,2020-09-01,TIRAMISU CAKE SLICE T/O,Remove,Remove,Remove


In [8]:
# To remove the rows of data that aren't the selected products and have the name Remove 
aldeburgh = aldeburgh[aldeburgh['ProductName']!='Remove']
southwold = southwold[southwold['ProductName']!='Remove']
darsham = darsham[darsham['ProductName']!='Remove']
norwich = norwich[norwich['ProductName']!='Remove']

In [9]:
# To keep the columns require 
aldeburgh = aldeburgh[['TransactionId', 'ProductName', 'Quantity']]
southwold = southwold[['TransactionId', 'ProductName', 'Quantity']]
darsham = darsham[['TransactionId', 'ProductName', 'Quantity']]
norwich = norwich[['TransactionId', 'ProductName', 'Quantity']]

In [10]:
# To validate the dat is in the correct format 
aldeburgh.head()

,TransactionId,ProductName,Quantity
1,325399103,Coffee,1
5,325403315,Madagascan Brownie,4
8,325407468,Madagascan Brownie,1
11,325413684,Tea,2
12,325413684,Friand,1


In [11]:
# To dropna values 
aldeburgh.dropna(inplace=True)
southwold.dropna(inplace=True)
darsham.dropna(inplace=True)
norwich.dropna(inplace=True)

# Aldeburgh Basket Analysis 

In [12]:
# Copy is taken for audit trail purposes 
aldeburgh_encoded = aldeburgh.copy()

In [13]:
aldeburgh_encoded = bakery.transaction_encoder(aldeburgh_encoded)

In [14]:
# Validate it worked 
aldeburgh_encoded.head()

,Almond Toast,Baguette,Bakewell,Cheese & Tomato Melt,Cheese Scone,Cheese Straw,Cinnamon Swirl,Coffee,Croissant,Croque Monsieur,Danish,Empanada,Friand,Fruit Scone,Madagascan Brownie,Magpie Sourdough,Moroccan Vegan Roll,Pain Au Chocolate,Sausage Roll,Tea
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True,False,False


In [15]:
a = apriori(aldeburgh_encoded, min_support=0.007, use_colnames=True)
association_rules(a, metric='lift', min_threshold=1).sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
16,(Croissant),(Pain Au Chocolate),0.076998,0.059748,0.021828,0.283495,4.744871,0.017228,1.312276
17,(Pain Au Chocolate),(Croissant),0.059748,0.076998,0.021828,0.365344,4.744871,0.017228,1.454335
30,"(Coffee, Croissant)",(Pain Au Chocolate),0.036497,0.059748,0.009157,0.250902,4.199361,0.006977,1.255180
33,(Pain Au Chocolate),"(Coffee, Croissant)",0.059748,0.036497,0.009157,0.153263,4.199361,0.006977,1.137901
32,(Croissant),"(Coffee, Pain Au Chocolate)",0.076998,0.031591,0.009157,0.118927,3.764529,0.006725,1.099124
31,"(Coffee, Pain Au Chocolate)",(Croissant),0.031591,0.076998,0.009157,0.289860,3.764529,0.006725,1.299747
13,(Pain Au Chocolate),(Cinnamon Swirl),0.059748,0.086502,0.016154,0.270370,3.125579,0.010986,1.252002
12,(Cinnamon Swirl),(Pain Au Chocolate),0.086502,0.059748,0.016154,0.186746,3.125579,0.010986,1.156161
26,"(Cinnamon Swirl, Coffee)",(Pain Au Chocolate),0.045375,0.059748,0.007856,0.173131,2.897692,0.005145,1.137123
29,(Pain Au Chocolate),"(Cinnamon Swirl, Coffee)",0.059748,0.045375,0.007856,0.131481,2.897692,0.005145,1.099142


In [16]:
a.sort_values('support', ascending=False)

,support,itemsets
7,0.592467,(Coffee)
18,0.144106,(Sausage Roll)
19,0.131587,(Tea)
15,0.090143,(Magpie Sourdough)
6,0.086502,(Cinnamon Swirl)
2,0.082672,(Bakewell)
8,0.076998,(Croissant)
46,0.072087,"(Tea, Coffee)"
14,0.062156,(Madagascan Brownie)
17,0.059748,(Pain Au Chocolate)


**UPDATE** comments and graphs and recommendations 

# Southwold Basket Analysis

In [17]:
# Copy is taken for audit trail purposes 
southwold_encoded = southwold.copy()

In [18]:
southwold_encoded = bakery.transaction_encoder(southwold_encoded)

In [19]:
# To validate it worked
southwold_encoded

,Almond Toast,Baguette,Bakewell,Cheese & Tomato Melt,Cheese Scone,Cheese Straw,Cinnamon Swirl,Coffee,Croissant,Croque Monsieur,Danish,Empanada,Friand,Fruit Scone,Madagascan Brownie,Magpie Sourdough,Moroccan Vegan Roll,Pain Au Chocolate,Sausage Roll,Tea
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165014,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
165015,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
165016,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
165017,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [20]:
s = apriori(southwold_encoded, min_support=0.007, use_colnames=True)
association_rules(s, metric='lift', min_threshold=1).sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
43,(Pain Au Chocolate),"(Cinnamon Swirl, Croissant)",0.068901,0.019773,0.008672,0.125858,6.364965,0.007309,1.121358
38,"(Cinnamon Swirl, Croissant)",(Pain Au Chocolate),0.019773,0.068901,0.008672,0.438553,6.364965,0.007309,1.658393
39,"(Cinnamon Swirl, Pain Au Chocolate)",(Croissant),0.020337,0.087814,0.008672,0.426400,4.855716,0.006886,1.590284
42,(Croissant),"(Cinnamon Swirl, Pain Au Chocolate)",0.087814,0.020337,0.008672,0.098751,4.855716,0.006886,1.087006
21,(Pain Au Chocolate),(Croissant),0.068901,0.087814,0.026009,0.377485,4.298677,0.019959,1.465323
20,(Croissant),(Pain Au Chocolate),0.087814,0.068901,0.026009,0.296184,4.298677,0.019959,1.322929
47,(Pain Au Chocolate),"(Coffee, Croissant)",0.068901,0.038935,0.011302,0.164028,4.212881,0.008619,1.149638
44,"(Coffee, Croissant)",(Pain Au Chocolate),0.038935,0.068901,0.011302,0.290272,4.212881,0.008619,1.311910
46,(Croissant),"(Coffee, Pain Au Chocolate)",0.087814,0.035075,0.011302,0.128701,3.669323,0.008222,1.107455
45,"(Coffee, Pain Au Chocolate)",(Croissant),0.035075,0.087814,0.011302,0.322218,3.669323,0.008222,1.345840


In [21]:
s.sort_values('support', ascending=False)

,support,itemsets
7,0.517480,(Coffee)
18,0.165211,(Sausage Roll)
15,0.112823,(Magpie Sourdough)
6,0.103085,(Cinnamon Swirl)
8,0.087814,(Croissant)
...,...,...
36,0.007545,"(Coffee, Croque Monsieur)"
37,0.007460,"(Coffee, Danish)"
27,0.007436,"(Sausage Roll, Cheese Scone)"
52,0.007417,"(Sausage Roll, Friand)"


**UPDATE comments**

# Darsham Basket Analysis 

In [22]:
# Copy taken for audit trail purposes
darsham_encoded = darsham.copy()

In [23]:
darsham_encoded = bakery.transaction_encoder(darsham_encoded)

In [24]:
# To validate it worked
darsham_encoded.head()

,Almond Toast,Baguette,Bakewell,Cheese & Tomato Melt,Cheese Scone,Cheese Straw,Cinnamon Swirl,Coffee,Croissant,Croque Monsieur,Danish,Empanada,Friand,Fruit Scone,Madagascan Brownie,Magpie Sourdough,Moroccan Vegan Roll,Pain Au Chocolate,Sausage Roll,Tea
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False
1,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
3,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False


In [25]:
d = apriori(darsham_encoded, min_support=0.007, use_colnames=True)
association_rules(d, metric='lift', min_threshold=1).sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
34,(Croissant),(Pain Au Chocolate),0.078016,0.063393,0.017270,0.221369,3.492030,0.012325,1.202890
35,(Pain Au Chocolate),(Croissant),0.063393,0.078016,0.017270,0.272434,3.492030,0.012325,1.267216
56,"(Coffee, Croissant)",(Pain Au Chocolate),0.039496,0.063393,0.007871,0.199282,3.143606,0.005367,1.169709
59,(Pain Au Chocolate),"(Coffee, Croissant)",0.063393,0.039496,0.007871,0.124161,3.143606,0.005367,1.096667
58,(Croissant),"(Coffee, Pain Au Chocolate)",0.078016,0.035305,0.007871,0.100889,2.857643,0.005117,1.072943
...,...,...,...,...,...,...,...,...,...
12,(Cinnamon Swirl),(Cheese Straw),0.096232,0.110035,0.010881,0.113067,1.027553,0.000292,1.003418
30,(Coffee),(Pain Au Chocolate),0.550799,0.063393,0.035305,0.064098,1.011119,0.000388,1.000753
31,(Pain Au Chocolate),(Coffee),0.063393,0.550799,0.035305,0.556923,1.011119,0.000388,1.013822
55,(Cinnamon Swirl),"(Sausage Roll, Coffee)",0.096232,0.081593,0.007902,0.082119,1.006448,0.000051,1.000573


In [26]:
d.sort_values('support', ascending=False)

,support,itemsets
7,0.550799,(Coffee)
18,0.193519,(Sausage Roll)
15,0.129913,(Magpie Sourdough)
5,0.110035,(Cheese Straw)
19,0.098800,(Tea)
...,...,...
79,0.007635,"(Cinnamon Swirl, Coffee, Pain Au Chocolate)"
34,0.007382,"(Cheese Straw, Cheese Scone)"
21,0.007319,"(Cheese Straw, Baguette)"
39,0.007272,"(Cheese Straw, Croissant)"


UPDATE COMMENTS

# Norwich Basket Analysis

In [27]:
# Copy of the dataset taken for audit trail purposes
norwich_encoded = norwich.copy()

In [28]:
norwich_encoded = bakery.transaction_encoder(norwich_encoded)

In [29]:
# To validate it worked
norwich_encoded.head()

,Almond Toast,Baguette,Bakewell,Cheese & Tomato Melt,Cheese Scone,Cheese Straw,Cinnamon Swirl,Coffee,Croissant,Croque Monsieur,Danish,Empanada,Friand,Fruit Scone,Madagascan Brownie,Magpie Sourdough,Moroccan Vegan Roll,Pain Au Chocolate,Sausage Roll,Tea
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [30]:
n = apriori(darsham_encoded, min_support=0.007, use_colnames=True)
association_rules(n, metric='lift', min_threshold=1).sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
34,(Croissant),(Pain Au Chocolate),0.078016,0.063393,0.017270,0.221369,3.492030,0.012325,1.202890
35,(Pain Au Chocolate),(Croissant),0.063393,0.078016,0.017270,0.272434,3.492030,0.012325,1.267216
56,"(Coffee, Croissant)",(Pain Au Chocolate),0.039496,0.063393,0.007871,0.199282,3.143606,0.005367,1.169709
59,(Pain Au Chocolate),"(Coffee, Croissant)",0.063393,0.039496,0.007871,0.124161,3.143606,0.005367,1.096667
58,(Croissant),"(Coffee, Pain Au Chocolate)",0.078016,0.035305,0.007871,0.100889,2.857643,0.005117,1.072943
...,...,...,...,...,...,...,...,...,...
12,(Cinnamon Swirl),(Cheese Straw),0.096232,0.110035,0.010881,0.113067,1.027553,0.000292,1.003418
30,(Coffee),(Pain Au Chocolate),0.550799,0.063393,0.035305,0.064098,1.011119,0.000388,1.000753
31,(Pain Au Chocolate),(Coffee),0.063393,0.550799,0.035305,0.556923,1.011119,0.000388,1.013822
55,(Cinnamon Swirl),"(Sausage Roll, Coffee)",0.096232,0.081593,0.007902,0.082119,1.006448,0.000051,1.000573


In [31]:
n.sort_values('support', ascending=False)

,support,itemsets
7,0.550799,(Coffee)
18,0.193519,(Sausage Roll)
15,0.129913,(Magpie Sourdough)
5,0.110035,(Cheese Straw)
19,0.098800,(Tea)
...,...,...
79,0.007635,"(Cinnamon Swirl, Coffee, Pain Au Chocolate)"
34,0.007382,"(Cheese Straw, Cheese Scone)"
21,0.007319,"(Cheese Straw, Baguette)"
39,0.007272,"(Cheese Straw, Croissant)"


Comments update

# Summary

# Recommendations

>[Return to Contents](#Contents)